# Prepare data to support DEM comparison - Tweed Heads

Various DEM/DSM and elevation values are used in the workflow. 
It's important to understand consistency between the DEM values.

DEMs to be compared

* NSW Spatial Services 1m - 2020 - "GDA2020 / MGA zone 55" ("EPSG:7855"), AHD71 - using local Geoid model
* ~~AGO 1m - Tamar 2008 - "GDA94 / MGA zone 55"~~
* ~~AGO 7cm (DSM) - 2021 - "WGS 84 / UTM zone 55S" ("EPSG:32755)~~

Comparison methods to be considered

* Comparison to surveyed ground control points
* Comparison of downsampled minimum elevation
* Comparison of points over road

In [39]:
import geopandas as gpd

### Ground control points

NSW Survey Control Information Management System (SCIMS) FeatureServer: https://portal.spatial.nsw.gov.au/server/rest/services/SurveyMarkGDA2020/FeatureServer

In [40]:
# constrain using building dataset boundary
buildings_path = "/home/lng/sharepoint/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/overture/tweed_heads_building.gpkg"

buildings  = gpd.read_file(buildings_path)

In [41]:
# Exported from NSW SCIMS Feature Server
gpkg_path = "/home/lng/sharepoint/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/Ground Control/NSW/tweed_heads_scims_marks.gpkg"
gdf = gpd.read_file(gpkg_path)


In [42]:
# Project to GDA2020 MGA55
gdf = gdf.to_crs(7856)


In [43]:
# Inspect data
gdf.marksymbol_label.unique()

array(['PM - Established GDA2020 and Accurate AHD71',
       'SS - Established GDA2020',
       'SS - Established GDA2020 and Accurate AHD71',
       'CP - Established GDA2020', 'PM - Established GDA2020',
       'CR - Established GDA2020',
       'TS - Established GDA2020 and Accurate AHD71',
       'GB - Established GDA2020 and Accurate AHD71',
       'TS - Established GDA2020',
       'MM - Established GDA2020 and Accurate AHD71',
       'CR - Established GDA2020 and Accurate AHD71',
       'MM - Established GDA2020', 'PM - Unknown', 'SS - Unknown',
       'SS - Accurate AHD71', 'PM - Accurate AHD71', 'TS - Unknown',
       'SS - Approximate GDA2020 Only',
       'SS - Accurate AHD71 and Approximate GDA2020', 'CP - Unknown',
       'MM - Unknown'], dtype=object)

In [44]:
gdf.monumenttype.unique()

array(['UNKNOWN', 'SPIKE IN ROAD', 'SURVEY PLAQUE IN CONC',
       'S/S PIN IN C/BOX', 'DH & WING IN KERB', 'BRASS PLAQUE',
       'SSM IN CONC', 'CONC PILLAR', 'DEEP DRIVEN ROD', 'LIGHTHOUSE',
       'STAR PICKET IN C/BOX', 'SSM TYPE 2', 'PLAQUE', 'SURVEY PLAQUE',
       'S/S PIN IN CONC (ORIG)', 'BRASS TRIANGLE IN CONC', 'STEEL PILLAR',
       'ANTENNA REFERENCE', 'SSM', 'SSM TYPE 15', 'S/S PIN',
       'S/S PIN (ORIG)', 'BOLT', 'DRILL HOLE IN CONC', 'SSM TYPE 1'],
      dtype=object)

Position quality criteria
* markstatus ="F" (found), "R" (restricted)  and
* gdaclass in "3A", "2A", "A", "B", "C", "D", "E" (established survey marks)
* ahdclass in "L2A", "LA", "LB", "LC", "LD", "LE", "2A", "A", "B" (accurate heights)


In [45]:
# Filter using above rules
filter = gdf.markstatus.isin(["F", "R"])
filter &= gdf.gdaclass.isin(["3A", "2A", "A", "B", "C", "D"])
filter &= gdf.ahdclass.isin(["L2A", "LA", "LB", "LC", "LD", "LE", "2A", "A", "B"])
gdf_filtered = gdf[filter]
gdf_filtered.monumenttype.unique()


array(['S/S PIN IN C/BOX', 'UNKNOWN', 'BRASS PLAQUE', 'SSM IN CONC',
       'DEEP DRIVEN ROD', 'LIGHTHOUSE', 'S/S PIN IN CONC (ORIG)',
       'BRASS TRIANGLE IN CONC', 'ANTENNA REFERENCE', 'STEEL PILLAR',
       'CONC PILLAR', 'DRILL HOLE IN CONC', 'STAR PICKET IN C/BOX'],
      dtype=object)

In [46]:
len(gdf_filtered)

63

Mark location criteria

* unknown
* c/box (below ground)
* antenna (above ground)
* pillar (above ground)
* lighthouse


In [47]:
# Filter out unreliable, not at ground level
filter = ["unknown", "c/box", "antenna", "pillar", "lighthouse"]

gdf_filtered = gdf_filtered[~gdf_filtered.monumenttype.str.lower().str.contains('|'.join(filter))]

len(gdf_filtered)

25

In [48]:
gdf_filtered.marksymbol_label.unique()

array(['SS - Established GDA2020 and Accurate AHD71',
       'GB - Established GDA2020 and Accurate AHD71',
       'PM - Established GDA2020 and Accurate AHD71',
       'SS - Established GDA2020',
       'TS - Established GDA2020 and Accurate AHD71'], dtype=object)

In [49]:
gdf_filtered.monumenttype.unique()

array(['BRASS PLAQUE', 'SSM IN CONC', 'DEEP DRIVEN ROD',
       'S/S PIN IN CONC (ORIG)', 'BRASS TRIANGLE IN CONC',
       'DRILL HOLE IN CONC'], dtype=object)

In [50]:
gdf_filtered.monumenttype.describe()

count               25
unique               6
top       BRASS PLAQUE
freq                10
Name: monumenttype, dtype: object

In [51]:
gdf_filtered.ahdheight_label = gdf_filtered.ahdheight_label.astype(float)

In [52]:
# Save as GeoPackage
gdf_filtered.to_file("/home/lng/sharepoint/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/Ground Control/NSW/tweed_heads_scims_marks_filtered.gpkg")